In [10]:
import os
os.chdir("/Users/mammoc/Desktop/Python")
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn import metrics
from sklearn import tree
airport = pd.read_csv("airport_original1.csv", sep = ';', header = 0)
#airport

In [11]:
# Dropping rows where 'Destination' is 4
airport = airport[airport['Destination'] != 4]
airport = airport.dropna(subset=['Destination'])

In [12]:
#FILL UP MISSING VALUES
#Age has 1 missing value (ID 214) => Replace by average age of 40
airport.loc[airport['ID'] == 214, 'Age'] = 40

#Gender has 3 missing values (ID 179, 199, 259)
# => Fill up by most frequency category - female - 2.0
airport.loc[airport['ID'].isin([179, 199, 259]), 'Gender'] = 2.0


#SeatClass has 3 missing value 
#Fill up by most frequency category = 1.0
airport.loc[airport['ID'].isin([245, 255, 450, 475]), 'SeatClass'] = 1.0

#AccessTime
AccessTime_by_ProvinceResidence = airport.groupby('ProvinceResidence')['AccessTime'].mean()
airport['AccessTime'] = airport.apply(
    lambda row: AccessTime_by_ProvinceResidence[row['ProvinceResidence']] if pd.isnull(row['AccessTime']) 
    else row['AccessTime'],axis=1)


# Recategorize 'Airline' into binary categories: 'Legacy' and 'Other'
airport['Airline[Legacy]'] = airport['Airline'].isin([1, 2]).astype(int)

# Calculate the mean 'Airfare' for each combination of 'Airline[Legacy]','Destination'
Airfare_by_Airline_Destination = airport.groupby(['Airline[Legacy]','Destination'])['Airfare'].mean()

#Fill the missing values in 'Airfare' with the average values calculated above
airport['Airfare'] = airport.apply(
    lambda row: Airfare_by_Airline_Destination[row['Airline[Legacy]'], row['Destination']] if pd.isnull(row['Airfare']) 
    else row['Airfare'], axis=1)

In [14]:
#---------------------- AIRPORT ----------------------
X = airport[['Age',
             'Gender',
             'TripPurpose',
             'Nationality',
             'TripDuration',
             'FlyingCompanion',
             'ProvinceResidence',
             'GroupTravel',
             'Destination',
             'DepartureTime',            
             'AccessTime',
             'NoTransport',
             'NoTripsLastYear',
             'SeatClass',
             'ModeTransport',
             'Airfare'
             ]]
#Seoul,Incheon and other, Incheon,Kyungki-do and others
y = airport['Airport']
random_state=109

clf = tree.DecisionTreeClassifier(class_weight=None, criterion='gini',
max_depth=6, max_features=None, max_leaf_nodes=None,
min_samples_leaf=10, min_samples_split=2,
min_weight_fraction_leaf=0.0, random_state=100, splitter='best')
clf = clf.fit(X, y)

In [36]:
#dot_data = tree.export_graphviz(clf, out_file='final.ap.Tree.dot', feature_names=X.columns)

In [23]:
#---------- 10-FOLD CROSS VALIDATION - AIRPORT ------------
from sklearn.model_selection import cross_val_score
# Perform 10-fold cross-validation and compute the accuracy
cv_scores = cross_val_score(clf, X, y, cv=10)

# Calculate the average accuracy across all 10 folds
average_accuracy = cv_scores.mean()

cv_recall_scores = cross_val_score(clf, X, y, cv=10, scoring='recall_macro')  # Use 'recall' for binary classification
# Calculate the average recall across all 10 folds
average_recall = cv_recall_scores.mean()

# Perform 10-fold cross-validation and compute the precision
cv_precision_scores = cross_val_score(clf, X, y, cv=10, scoring='precision_macro')  # Use 'precision' for binary classification
# Calculate the average precision across all 10 folds
average_precision = cv_precision_scores.mean()

print(f'Average accuracy from 10-fold cross-validation: {average_accuracy:.2f}')
print(f'Average recall from 10-fold cross-validation: {average_recall:.2f}')
print(f'Average precision from 10-fold cross-validation: {average_precision:.2f}')

Average accuracy from 10-fold cross-validation: 0.84
Average recall from 10-fold cross-validation: 0.83
Average precision from 10-fold cross-validation: 0.84


In [19]:
#-------------- AIRLINE ----------------
airport['Airline[Legacy]'] = (airport['Airline'].isin([1,2])).astype(int)
X1 = airport[['Age',
             'Gender',
             'TripPurpose',
             'Occupation',
             'Nationality',
             'TripDuration',
             'FlyingCompanion',
             'ProvinceResidence',
             'GroupTravel',
             'Destination',
             'DepartureTime',            
             'AccessTime',
             'NoTransport',
             'NoTripsLastYear',
             'SeatClass',
             'ModeTransport',
             'Airfare'
             ]]
#Seoul,Incheon and other, Incheon,Kyungki-do and others
y1 = airport['Airline[Legacy]']
random_state=109

clf1 = tree.DecisionTreeClassifier(class_weight=None, criterion='gini',
max_depth=5, max_features=None, max_leaf_nodes=None,
min_samples_leaf=10, min_samples_split=2,
min_weight_fraction_leaf=0.0, random_state=100, splitter='best')
clf1 = clf1.fit(X1, y1)

In [50]:
#dot_data1 = tree.export_graphviz(clf1, out_file='final.al.Tree.dot', feature_names=X1.columns)

In [24]:
#---------- 10-FOLD CROSS VALIDATION - AIRLINE ------------
# Perform 10-fold cross-validation and compute the accuracy
cv_scores = cross_val_score(clf1, X1, y1, cv=10)

# Calculate the average accuracy across all 10 folds
average_accuracy = cv_scores.mean()

cv_recall_scores1 = cross_val_score(clf1, X, y, cv=10, scoring='recall_macro')  # Use 'recall' for binary classification
# Calculate the average recall across all 10 folds
average_recall = cv_recall_scores1.mean()

# Perform 10-fold cross-validation and compute the precision
cv_precision_scores1 = cross_val_score(clf1, X, y, cv=10, scoring='precision_macro')  # Use 'precision' for binary classification
# Calculate the average precision across all 10 folds
average_precision = cv_precision_scores1.mean()

print(f'Average accuracy from 10-fold cross-validation: {average_accuracy:.2f}')
print(f'Average recall from 10-fold cross-validation: {average_recall:.2f}')
print(f'Average precision from 10-fold cross-validation: {average_precision:.2f}')

Average accuracy from 10-fold cross-validation: 0.68
Average recall from 10-fold cross-validation: 0.81
Average precision from 10-fold cross-validation: 0.82
